## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.layers import Input, Dense
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [ ]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [ ]:
# Create a list of at least 10 column names to use as X data
X_data = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'HourlyRate', 'NumCompaniesWorked', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'StockOptionLevel']


# Create X_df using your selected columns
X_df = attrition_df[X_data]

# Show the data types for X_df
X_df.dtypes


Education                  int64
Age                        int64
DistanceFromHome           int64
JobSatisfaction            int64
HourlyRate                 int64
NumCompaniesWorked         int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
StockOptionLevel           int64
dtype: object

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# The X data is currently in the int64 data type

In [46]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [47]:
# Create a OneHotEncoder for the Department column

dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])
# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoded = dept_encoder.transform(y_test[['Department']])

In [48]:
# Create a OneHotEncoder for the Attrition column

attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoded = attr_encoder.transform(y_train[['Attrition']])
y_test_attr_encoded = attr_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [49]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer_1 = Dense(10, activation='relu')(input_layer)
shared_layer_2 = Dense(5, activation='relu')(shared_layer_1)

In [51]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer

dept_hidden_layer = Dense(16, activation='relu')(shared_layer_2)
# Create the output layer

dept_output_layer = Dense(y_train_dept_encoded.shape[1], activation='sigmoid', name='department_output')(dept_hidden_layer)



In [52]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer

attrition_hidden_layer = Dense(16, activation='relu')(shared_layer_2)
# Create the output layer

attrition_output_layer = Dense(y_train_attr_encoded.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden_layer)

In [53]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense_20 (Dense)            (None, 10)                   110       ['input_6[0][0]']             
                                                                                                  
 dense_21 (Dense)            (None, 5)                    55        ['dense_20[0][0]']            
                                                                                                  
 dense_22 (Dense)            (None, 16)                   96        ['dense_21[0][0]']            
                                                                                            

In [54]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_dept_encoded, 'attrition_output': y_train_attr_encoded},
                    epochs=100, batch_size=32, validation_split=0.2, verbose=2)

Epoch 1/100
28/28 - 3s - loss: 1.5434 - department_output_loss: 0.9542 - attrition_output_loss: 0.5892 - department_output_accuracy: 0.6663 - attrition_output_accuracy: 0.7832 - val_loss: 1.4327 - val_department_output_loss: 0.8876 - val_attrition_output_loss: 0.5452 - val_department_output_accuracy: 0.6606 - val_attrition_output_accuracy: 0.8235 - 3s/epoch - 115ms/step
Epoch 2/100
28/28 - 0s - loss: 1.3958 - department_output_loss: 0.8733 - attrition_output_loss: 0.5225 - department_output_accuracy: 0.6674 - attrition_output_accuracy: 0.8286 - val_loss: 1.3360 - val_department_output_loss: 0.8257 - val_attrition_output_loss: 0.5103 - val_department_output_accuracy: 0.6606 - val_attrition_output_accuracy: 0.8235 - 109ms/epoch - 4ms/step
Epoch 3/100
28/28 - 0s - loss: 1.3295 - department_output_loss: 0.8308 - attrition_output_loss: 0.4987 - department_output_accuracy: 0.6674 - attrition_output_accuracy: 0.8286 - val_loss: 1.2903 - val_department_output_loss: 0.7900 - val_attrition_outpu

In [55]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled,
                         {'department_output': y_test_dept_encoded, 'attrition_output': y_test_attr_encoded},
                         verbose=2)


12/12 - 0s - loss: 1.2382 - department_output_loss: 0.8533 - attrition_output_loss: 0.3849 - department_output_accuracy: 0.6168 - attrition_output_accuracy: 0.8478 - 140ms/epoch - 12ms/step


In [57]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {results[3]*100}")
print(f"Attrition Accuracy: {results[4]*100}")

Department Accuracy: 61.684781312942505
Attrition Accuracy: 84.78260636329651


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1..Accuracy is not the best method to use on this data. There is a huge difference in the accuracy between the Attrition and Department accuracy. This may denote excessive false positives and false negatives. Measuring Precison and Recall may provide more insight.
2.I tried both Softmax and Sigmoid. I went with Sigmoid because it raised the accuracy of the Attrition output from 81% to 85% and it raised the accuracy of the Department output from 55% to 62%.
3.Make sure the classes are balanced. Try different feature engineering techniques. Add more hidden layers, change the number of neurons in the layers, or try different activation functions.